In [4]:
import json

from collections import defaultdict

In [5]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r", encoding="utf-8") as file:
    postData = json.load(file)
    
with open("../DataProcessing/TestData/runeSheet.json", "r", encoding="utf-8") as file:
    runeSheet = json.load(file)
    
participantsData = data['metadata']['participants']
framesData = data['info']['frames']
participantFrames = [frame['participantFrames'] for frame in framesData]

championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []
participantChampionMapping = {}


In [6]:
targetPuuid = '7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg'

In [7]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)
    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]
    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)
findParticipantData(data)

def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")

Participant ID: 2, Champion: MasterYi


In [8]:
def extractBuildingKillData(data, participantData):
    buildingKillData = []

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', None)
                buildingType = event.get('buildingType', 'Unknown Building Type')
                timestamp = event['timestamp']
                currentMinute = round(timestamp / 60000, 2)
                position = event['position']
                laneType = event['laneType']
                killerId = event['killerId']
                assistingParticipantIds = event.get('assistingParticipantIds', [])

                # Retrieve champion name for the killer
                killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                  f"Minions")

                # Retrieve champion names for assisting participants only if there are assisting participants
                assistingChampionNames = [next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}") for assistId in assistingParticipantIds]

                towerOrBuildingType = towerType if towerType in ['OUTER_TURRET', 'INNER_TURRET'] else None
                
                buildingKillData.append({
                    'teamId': teamId,
                    'towerOrBuildingType': towerOrBuildingType,
                    'timestamp': currentMinute,
                    'position': position,
                    'laneType': laneType,
                    'killerId': killerId,
                    'killerName': killerName,
                    'assistingParticipantIds': assistingParticipantIds,
                    'assistingChampionNames': assistingChampionNames
                })

    return buildingKillData

# Example usage:
buildingKillData = extractBuildingKillData(data, participantData)

sortedBuildingKillData = sorted(buildingKillData, key=lambda x: (x['teamId'], x['timestamp']))

# Display the extracted data
# Display the extracted data
for kill in sortedBuildingKillData:
    currentMinute = kill['timestamp']
    towerOrBuildingType = kill['towerOrBuildingType']
    
    # Check if towerOrBuildingType is 'OUTER_TURRET' or 'INNER_TURRET' before printing
    if towerOrBuildingType in ['OUTER_TURRET', 'INNER_TURRET']:
        print(f"Team: {kill['teamId']}, Tower Type: {towerOrBuildingType}, Timestamp: {currentMinute}, Lane: {kill['laneType']}")
        print(f"Killer: {kill['killerName']}")
    
        # Print assisting champions only if there are assisting participants
        if kill['assistingParticipantIds']:
            print(f"Assisting Champions: {kill['assistingChampionNames']}")
    
        print('\n')



Team: 100, Tower Type: OUTER_TURRET, Timestamp: 20.04, Lane: BOT_LANE
Killer: Ezreal
Assisting Champions: ['Nautilus']


Team: 100, Tower Type: OUTER_TURRET, Timestamp: 20.91, Lane: MID_LANE
Killer: Sylas


Team: 100, Tower Type: OUTER_TURRET, Timestamp: 29.4, Lane: TOP_LANE
Killer: Minions


Team: 100, Tower Type: INNER_TURRET, Timestamp: 29.9, Lane: TOP_LANE
Killer: Minions
Assisting Champions: ['Shen']


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 17.34, Lane: MID_LANE
Killer: Veigar


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 20.36, Lane: TOP_LANE
Killer: Jax


Team: 200, Tower Type: INNER_TURRET, Timestamp: 21.02, Lane: TOP_LANE
Killer: Jax


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 24.95, Lane: BOT_LANE
Killer: MasterYi


Team: 200, Tower Type: INNER_TURRET, Timestamp: 28.63, Lane: MID_LANE
Killer: Varus


Team: 200, Tower Type: INNER_TURRET, Timestamp: 37.34, Lane: BOT_LANE
Killer: Minions


